In [5]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
%matplotlib inline
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna
import oBehave.behavior_anaylsis as obba
import sklearn.decomposition as decomp

import time

from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

experiment_id = 639438856
manifest = obhf.load_manifest()
obba.includeNovelSession(manifest)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,experiment_id,experiment_container_id,targeted_structure,imaging_depth,cre_line,reporter_line,full_genotype,session_type,donor_id,specimen_id,experiment_date,experiment_datetime,first_session
0,644942849,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_A,329071,596175193,10/24/2017 10:32,2017-10-24 10:00:32,True
1,645035903,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_B,329071,596175193,10/25/2017 10:34,2017-10-25 10:00:34,True
2,645086795,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_C,329071,596175193,10/26/2017 10:27,2017-10-26 10:00:27,True
3,645362806,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_D,329071,596175193,10/30/2017 11:41,2017-10-30 11:00:41,True
4,646922970,646555436,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_A,330982,596393048,11/7/2017 14:19,2017-11-07 14:00:19,True
5,647108734,646555436,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_B,330982,596393048,11/8/2017 11:56,2017-11-08 11:00:56,True
6,647551128,646555436,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_C,330982,596393048,11/9/2017 11:48,2017-11-09 11:00:48,True
7,647887770,646555436,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_D,330982,596393048,11/10/2017 13:06,2017-11-10 13:00:06,True
8,648647430,648562391,VISal,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_A,330982,596393048,11/15/2017 11:58,2017-11-15 11:00:58,False
9,649118720,648562391,VISal,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_B,330982,596393048,11/16/2017 11:45,2017-11-16 11:00:45,False


In [7]:
def compute_lifetime_sparseness(image_responses):
    # image responses should be an array of the trial averaged responses to each image
    # sparseness = 1-(sum of trial averaged responses to images / N)squared / (sum of (squared mean responses / n)) / (1-(1/N))
    # N = number of images
    # Modified shamelessly from allen institute code
    N = float(len(image_responses))
    ls = ((1-(1/N) * ((np.power(np.abs(image_responses).sum(axis=0),2)) / (np.power(image_responses,2).sum(axis=0)))) / (1-(1/N)))
    return ls


keylist = ['experiment_id','cell_indices','cell_specimen_ids','session_type','cre_line',
           'SparAve','Spar1','Spar2','Spar3','Spar4','Spar5','Shit','Smiss']
cr = {k:[] for k in keylist}
for ex,row in manifest.iterrows():
    if not row['first_session']:
        continue
    print(row['experiment_id'])
    dataset= VisualBehaviorOphysDataset(experiment_id=row['experiment_id'], cache_dir=obhf.drive_path)
    analysis = ResponseAnalysis(dataset)
    FR = obba.includeAddRepeats(analysis.flash_response_df,dataset.all_trials)
    unique_cells = np.unique(FR['cell'])
    unique_images = np.unique(FR['image_name'])
    print(len(unique_cells))
    for ii in unique_cells:
        stime = time.time()
        cr['experiment_id'].append(row['experiment_id'])
        cr['cell_indices'].append(dataset.cell_indices[ii])
        cr['cell_specimen_ids'].append(dataset.cell_specimen_ids[ii])
        cr['cre_line'].append(row['cre_line'])
        cr['session_type'].append(row['session_type'])
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['SparAve'].append(compute_lifetime_sparseness(im_means))
        for rr in [1,2,3,4,5]:
            im_means = [FR[(FR['image_name']==image)&(FR['cell']==ii)&(FR['repeats']==rr)].mean_response.mean() for image in np.unique(FR['image_name'])]
            cr['Spar'+str(rr)].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='HIT')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Shit'].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='MISS')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Smiss'].append(compute_lifetime_sparseness(im_means))
        print(time.time()-stime)
CellResp = pd.DataFrame(cr)

644942849
loading trial response dataframe
loading flash response dataframe
12
0.699155092239
0.690745830536
0.691195011139
0.690325975418
0.691327810287
0.691833972931
0.690372943878
0.692614078522
0.690312862396
0.69141292572
0.690892934799
0.690749168396
645035903
loading trial response dataframe
loading flash response dataframe
11
0.656512975693
0.649142980576
0.649562120438
0.648685932159
0.649038076401


KeyboardInterrupt: 

In [8]:
manifest.shape

(60, 13)

In [10]:
60*100*0.7 / 60.

70.0